In [9]:
from pydantic import BaseModel
import requests
from datetime import datetime

class WeatherData(BaseModel):
    latitude: float
    longitude: float
    forecast_days: int = 3
    past_days: int = 7
    weather: dict = {}

    def fetch(self) -> None:
        """Fetch weather including FAO ET0 from Open-Meteo."""
        try:
            url = "https://api.open-meteo.com/v1/forecast"
            params = {
                "latitude": self.latitude,
                "longitude": self.longitude,
                "current_weather": True,
                "past_days": self.past_days,
                "forecast_days": self.forecast_days,
                "daily": (
                    "temperature_2m_max,temperature_2m_min,"
                    "precipitation_sum,wind_speed_10m_max,"
                    "relative_humidity_2m_mean,et0_fao_evapotranspiration"
                ),
                "timezone": "auto"
            }

            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            current = data.get("current_weather", {})
            daily = data.get("daily", {})
            times = daily.get("time", [])

            today = datetime.now().date().isoformat()

            past_summary = []
            forecast_summary = []

            for i, date in enumerate(times):
                day_summary = (
                    f"{date}: Temp {daily['temperature_2m_min'][i]}-{daily['temperature_2m_max'][i]}°C, "
                    f"Humidity {daily['relative_humidity_2m_mean'][i]}%, "
                    f"Rain {daily['precipitation_sum'][i]}mm, "
                    f"Wind Max {daily['wind_speed_10m_max'][i]} km/h, "
                    f"ETo {daily['et0_fao_evapotranspiration'][i]}mm"
                )
                if date < today:
                    past_summary.append(day_summary)
                else:
                    forecast_summary.append(day_summary)

            self.weather = {
                "location": f"Lat: {self.latitude}, Lon: {self.longitude}",
                "past_7_days": past_summary[-self.past_days:],
                "current": {
                    "temp": f"{current.get('temperature', 'N/A')}°C",
                    "wind": f"{current.get('windspeed', 'N/A')} km/h"
                },
                "forecast_next_days": forecast_summary[:self.forecast_days]
            }

        except requests.RequestException as e:
            self.weather = {"error": str(e)}
        except Exception as ex:
            self.weather = {"error": str(ex)}

# Example usage:
weather_instance = WeatherData(latitude=-1.2921, longitude=36.8219)
weather_instance.fetch()
print(weather_instance.weather)


{'location': 'Lat: -1.2921, Lon: 36.8219', 'past_7_days': ['2025-04-18: Temp 16.8-25.7°C, Humidity 75%, Rain 13.1mm, Wind Max 13.5 km/h, ETo 3.95mm', '2025-04-19: Temp 17.0-25.6°C, Humidity 77%, Rain 22.7mm, Wind Max 13.1 km/h, ETo 3.38mm', '2025-04-20: Temp 17.1-25.9°C, Humidity 78%, Rain 43.4mm, Wind Max 14.8 km/h, ETo 3.68mm', '2025-04-21: Temp 16.5-25.3°C, Humidity 77%, Rain 6.8mm, Wind Max 14.4 km/h, ETo 4.06mm', '2025-04-22: Temp 17.3-22.3°C, Humidity 82%, Rain 6.8mm, Wind Max 12.0 km/h, ETo 2.5mm', '2025-04-23: Temp 16.8-20.4°C, Humidity 85%, Rain 6.1mm, Wind Max 14.6 km/h, ETo 1.78mm', '2025-04-24: Temp 15.8-23.0°C, Humidity 74%, Rain 0.3mm, Wind Max 19.5 km/h, ETo 3.3mm'], 'current': {'temp': '17.2°C', 'wind': '6.2 km/h'}, 'forecast_next_days': ['2025-04-25: Temp 16.4-24.6°C, Humidity 72%, Rain 0.3mm, Wind Max 16.2 km/h, ETo 4.41mm', '2025-04-26: Temp 15.4-26.7°C, Humidity 69%, Rain 0.2mm, Wind Max 14.2 km/h, ETo 5.24mm', '2025-04-27: Temp 16.3-26.3°C, Humidity 70%, Rain 0.2mm

In [53]:
# Define the state schema
import os
import re
import json
import numpy as np
import io
from pydantic import BaseModel, Field
from typing import Dict, Any, List, Optional
from matplotlib import pyplot as plt
from ultralytics import YOLO
from langchain_core.runnables import RunnableLambda
from langchain_anthropic import ChatAnthropic
import boto3

from compute import full_image_processing_pipeline, read_image_from_s3, estimate_cwr_from_ndvi_and_weather, ndvi_to_kc


# S3 helper function for saving files
def save_to_s3(data, bucket_name, key, content_type=None):
    """
    Save data to an S3 bucket
    
    Parameters:
        data: The data to save (bytes or file-like object)
        bucket_name: Name of the S3 bucket
        key: S3 object key (path)
        content_type: MIME type of the data (optional)
    
    Returns:
        str: Full S3 URI of the saved object
    """
    s3_client = boto3.client(
        's3',
        aws_access_key_id=os.environ.get('AWS_ACCESS_KEY'),
        aws_secret_access_key=os.environ.get('AWS_SECRET_KEY'),
        region_name=os.environ.get('AWS_REGION')
    )
    
    extra_args = {}
    if content_type:
        extra_args['ContentType'] = content_type
    
    s3_client.put_object(Body=data, Bucket=bucket_name, Key=key, **extra_args)
    return f"s3://{bucket_name}/{key}"


class LocationModel(BaseModel):
    latitude: Optional[float] = None
    longitude: Optional[float] = None


class InputStateModel(BaseModel):
    user_id: Optional[str] = None
    image_key: str
    coordinates: Optional[LocationModel] = None


class WeatherDataOutputModel(BaseModel):
    location: Optional[str] = None
    past_7_days: List[str] = []
    current: Dict[str, Any] = {}
    forecast_next_days: List[str] = []


class YoloAnalysisOutputModel(BaseModel):
    status: Optional[str] = None
    class_labels: List[str] = []
    scores: List[float] = []
    bounding_boxes: List[List[float]] = []
    save_path: Optional[str] = None


class NDVIOutputModel(BaseModel):
    ndvi_summary: Optional[Dict[str, float]] = None
    save_path: Optional[str] = None


class MauiRecommendationModel(BaseModel):
    risk_level: str = "Unknown"
    advice: str = "No advice available"
    data_summary: Dict[str, Any] = Field(default_factory=dict)  # Changed to accept a dict


class OutputStateModel(BaseModel):
    user_id: Optional[str] = None
    yolo_result: Optional[YoloAnalysisOutputModel] = None
    weather_data: Optional[WeatherDataOutputModel] = None
    ndvi_result: Optional[NDVIOutputModel] = None
    recommendation: Optional[MauiRecommendationModel] = None


# — define the one "State" model that has *all* fields —
class State(InputStateModel, OutputStateModel):
    pass


def check_for_tiff(input_state: InputStateModel):
    if input_state.image_key.endswith(".tiff"):
        return "NDVI_pipeline"
    else:
        return "YOLO_analysis"


def get_today_eto(state: State) -> float:
    try:
        today_weather_str = state.weather_data.past_7_days[-1]  # e.g., "2025-04-24: Temp ..., ETo 3.3mm"
        eto_match = re.search(r'ETo\s([\d.]+)mm', today_weather_str)
        if eto_match:
            return float(eto_match.group(1))
    except Exception as e:
        print(f"Error extracting today's ETo: {e}")
    return 0.0  # fallback


def weather_node(input_state: State) -> WeatherDataOutputModel:
    weather_state = WeatherDataOutputModel()
    latitude = input_state.coordinates.latitude
    longitude = input_state.coordinates.longitude

    if latitude is not None and longitude is not None:
        weather = WeatherData(latitude=latitude, longitude=longitude)
        weather.fetch()  # this populates weather.weather

        if weather.weather:  # make sure it's populated
            weather_state = WeatherDataOutputModel(**weather.weather)

    return {"user_id": input_state.user_id, "weather_data": weather_state.model_dump()}


def YOLO_analysis(input_state: State) -> YoloAnalysisOutputModel:
    image_path = input_state.image_key
    image = read_image_from_s3("qijaniproductsbucket", image_path)
    yolo_state = YoloAnalysisOutputModel()
    output_bucket = "qijaniproductsbucket"  # Update with your output bucket name

    try:
        model = YOLO("yolo11s-pest-detection/best.pt")
        # For cloud deployment, we need to disable local saving
        results = model.predict(image, save=False)

        if not results:
            yolo_state = {"status": "No detections"}
            return yolo_state

        first_result = results[0]
        
        # Instead of saving locally, we'll render the annotated image and save to S3
        # Get the annotated image from the result
        annotated_img = first_result.plot()
        
        # Save the annotated image to S3
        img_bytes = io.BytesIO()
        plt.imsave(img_bytes, annotated_img, format='PNG')
        img_bytes.seek(0)
        
        # Generate S3 path
        base_filename = os.path.basename(image_path)
        s3_save_path = f"outputs/yolo_{base_filename}"
        
        # Save to S3
        s3_uri = save_to_s3(img_bytes, output_bucket, s3_save_path, content_type='image/png')

        yolo_state = {
            "status": "Success",
            "class_labels": first_result.names.values(),
            "scores": [float(c) for c in first_result.boxes.conf.tolist()],
            "bounding_boxes": [b.tolist() for b in first_result.boxes.xyxy],
            "save_path": s3_uri,
        }
        yolo_output = YoloAnalysisOutputModel(
            status=yolo_state["status"],
            class_labels=list(yolo_state["class_labels"]),  # convert dict_values to list
            scores=yolo_state["scores"],
            bounding_boxes=yolo_state["bounding_boxes"],
            save_path=yolo_state["save_path"]
        )

    except Exception as e:
        error_output = YoloAnalysisOutputModel(status=f"Error: {e}")
        return {"user_id": input_state.user_id, 
                "yolo_result": error_output.model_dump()}


    return {"user_id": input_state.user_id, "yolo_result": yolo_output.model_dump()}


def NDVI_analysis(input_state: State) -> NDVIOutputModel:
    RADIOMETRIC_PARAMS = {
        'gain': [0.012] * 5,
        'offset': [0] * 5,
        'sunelev': 60.0,
        'edist': 1.0,
        'Esun': [1913, 1822, 1557, 1317, 1074],
        'blackadjust': 0.01,
        'low_percentile': 1
    }
    output_bucket = "qijaniproductsbucket"  # Update with your output bucket name

    try:
        image = read_image_from_s3("qijaniproductsbucket", input_state.image_key)
        ndvi_noise_reduced, _, kc_map = full_image_processing_pipeline(
            image,
            RADIOMETRIC_PARAMS,
            detector_type='ORB',
            noise_method='median',
            noise_kernel_size=3,
            sigma=1.0,
            nir_band_index=4,
            red_band_index=2,
            visualize=False,
            use_parallel_noise_reduction=False
        )

        # Generate base filename for S3
        base_filename = os.path.basename(input_state.image_key)
        s3_image_path = f"outputs/ndvi_{base_filename}.jpg"
        s3_npy_path = f"outputs/ndvi_{base_filename}.npy"

        # Save NDVI plot to S3
        plt.figure(figsize=(10, 8))
        plt.imshow(ndvi_noise_reduced, cmap='RdYlGn')
        plt.colorbar(label='NDVI Value')
        plt.title("NDVI Analysis")
        
        # Save plot to bytes buffer instead of file
        img_buf = io.BytesIO()
        plt.savefig(img_buf, format='jpg', dpi=300)
        img_buf.seek(0)
        plt.close()
        
        # Save image to S3
        s3_image_uri = save_to_s3(img_buf, output_bucket, s3_image_path, content_type='image/jpeg')
        
        # Save numpy array to S3
        npy_buf = io.BytesIO()
        np.save(npy_buf, ndvi_noise_reduced)
        npy_buf.seek(0)
        s3_npy_uri = save_to_s3(npy_buf, output_bucket, s3_npy_path, content_type='application/octet-stream')

        cwr = estimate_cwr_from_ndvi_and_weather(ndvi_to_kc(ndvi_noise_reduced), get_today_eto(input_state))
        ndvi_summary = {
            "min": float(np.min(ndvi_noise_reduced)),
            "max": float(np.max(ndvi_noise_reduced)),
            "mean": float(np.mean(ndvi_noise_reduced)),
            "cwr": cwr
        }

        return {
            "user_id": input_state.user_id,
            "ndvi_result": NDVIOutputModel(ndvi_summary=ndvi_summary, save_path=s3_npy_uri).model_dump()
        }

    except Exception as e:
        return {
            "user_id": input_state.user_id,
            "ndvi_result": {"error": str(e)}
        }


def Maui(input_state: State) -> MauiRecommendationModel:
    model = ChatAnthropic(
        model_name="claude-3-haiku-20240307",
        temperature=0.3,
        api_key=os.getenv("ANTHROPIC_API_KEY")
    )

    ndvi_result = getattr(input_state, 'ndvi_result', None)
    ndvi_summary = getattr(ndvi_result, 'ndvi_summary', {'mean': 'No NDVI data'}) if ndvi_result else {'mean': 'No NDVI data'}
    ndvi_image = getattr(ndvi_result, 'save_path', 'No NDVI image saved') if ndvi_result else 'No NDVI image saved'

    yolo_detection = getattr(input_state, 'yolo_result', None)
    weather_summary = getattr(input_state, 'weather_data', {'summary': 'No weather data'})

    # Fix the context string - use double curly braces to escape them in f-strings
    context = f"""
    You are an intelligent agricultural assistant trained for precision farming.
    Use the following collected field data to internally reason in a scientific, agronomic way — analyze relationships between pest presence, vegetation health (NDVI), and current weather conditions (especially ETo and rainfall).
    However, when giving advice to the farmer, speak in plain, clear, and actionable terms — avoid technical jargon, and ensure your reasoning is simple and practical.
    
    Data provided:
    * Pest Detection: {', '.join([f"{label} ({score:.2f})" for label, score in zip(yolo_detection.class_labels, yolo_detection.scores)]) if yolo_detection else 'No pests detected'}
    * NDVI Summary: {ndvi_summary}
    * NDVI Image Path: {ndvi_image}
    * Weather Summary (Current): {weather_summary}
    
    Your output must be in the following JSON format:
    {{
        "risk_level": "Low|Moderate|High",
        "advice": "Actionable recommendation here",
        "data_summary": "Recap all findings in no more than 60 words"
    }}

    Make sure to follow this exact format, with these keys, and provide only valid JSON in your response.
    """
    
    # Use standard messages API instead of structured output
    response = model.invoke([{"role": "user", "content": context}])
    
    # Extract JSON response from the model output
    try:
        # Extract JSON content from response
        response_text = response.content
        # Find JSON content
        import json
        import re
        
        # Try to find JSON pattern in the response
        json_match = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
        else:
            # Try to find a JSON object directly
            json_match = re.search(r'(\{.*\})', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
            else:
                json_str = response_text
                
        recommendation_data = json.loads(json_str)
        
        # Accommodate the string data_summary by converting it to a dict if necessary
        data_summary = recommendation_data.get("data_summary", "")
        if isinstance(data_summary, str):
            data_summary = {"summary": data_summary}
            
        recommendation = MauiRecommendationModel(
            risk_level=recommendation_data.get("risk_level", "Unknown"),
            advice=recommendation_data.get("advice", "No advice available"),
            data_summary=data_summary
        )
    except Exception as e:
        # Fallback if JSON parsing fails
        recommendation = MauiRecommendationModel(
            risk_level="Unknown",
            advice=f"Error parsing response: {str(e)}",
            data_summary={"error": "Failed to parse model output"}
        )
    
    return {"user_id": input_state.user_id, "recommendation": recommendation}

In [54]:
from langgraph.graph import StateGraph, START, END


# Create the state graph
graph = StateGraph(State)

# Add nodes
graph.add_node("NDVI_pipeline", NDVI_analysis)
graph.add_node("YOLO_analysis", YOLO_analysis)
graph.add_node("weather_node", weather_node)
graph.add_node("Maui", Maui)

# Define edges and conditional branching
graph.add_edge(START, "weather_node")
graph.add_conditional_edges("weather_node", check_for_tiff, ["NDVI_pipeline", "YOLO_analysis"])  # Checks for TIFF and routes accordingly
graph.add_edge("NDVI_pipeline", "YOLO_analysis")
graph.add_edge("YOLO_analysis", "Maui")
graph.add_edge("Maui", END)

# Compile the graph
advisor_graph = graph.compile()

In [55]:
result = advisor_graph.invoke(
    State(
        user_id="xyz",
        coordinates=LocationModel(latitude=37.7749, longitude=-122.4194),
        image_key="Maize.jpg"
    )
)

# Check what's actually in the result
print("Result structure:", result)



0: 512x896 (no detections), 237.3ms
Speed: 3.4ms preprocess, 237.3ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 896)
Result structure: {'user_id': 'xyz', 'yolo_result': {'status': 'Success', 'class_labels': ['rice leaf roller', 'rice leaf caterpillar', 'paddy stem maggot', 'asiatic rice borer', 'yellow rice borer', 'rice gall midge', 'Rice Stemfly', 'brown plant hopper', 'white backed plant hopper', 'small brown plant hopper', 'rice water weevil', 'rice leafhopper', 'grain spreader thrips', 'rice shell pest', 'grub', 'mole cricket', 'wireworm', 'white margined moth', 'black cutworm', 'large cutworm', 'yellow cutworm', 'red spider', 'corn borer', 'army worm', 'aphids', 'Potosiabre vitarsis', 'peach borer', 'english grain aphid', 'green bug', 'bird cherry-oataphid', 'wheat blossom midge', 'penthaleus major', 'longlegged spider mite', 'wheat phloeothrips', 'wheat sawfly', 'cerodonta denticornis', 'beet fly', 'flea beetle', 'cabbage army worm', 'beet army worm', 'Beet spo

In [43]:
print("Risk Level:", result["recommendation"].risk_level)
print("Advice:", result["recommendation"].advice)
print("Summary:", result["recommendation"].data_summary)

Risk Level: Moderate
Advice: Monitor the field closely for any signs of pest activity. Consider applying a targeted, organic pest control measure if the pest population increases. Ensure the field is receiving adequate moisture through irrigation or natural rainfall to support healthy plant growth.
Summary: {'summary': 'Pest presence detected, but no NDVI data available to assess vegetation health. Recent weather conditions have been dry with moderate evapotranspiration, which could impact plant growth and make the crop more susceptible to pests.'}
